In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import datetime
from sklearn.model_selection import train_test_split

import torch
import re
import os # os는 파이썬의 내장 모듈 중 하나로, 운영체제와 상호 작용하기 위한 기능을 제공
import shutil # shutil은 파일 및 디렉토리 작업을 위한 파이썬의 내장 모듈 중 하나
import glob # glob은 파일 경로를 패턴 매칭을 통해 검색하는데 사용되는 파이썬의 내장 모듈 중 하나
import zipfile
import cv2

# 1. 데이터 통합 및 전처리

In [88]:
# 경로 설정
original_data_path = '../BoneAge/Original_Data/'
path = '../BoneAge/Data/'

# xlsx파일과 zip파일 경로 추출
xls_path = glob.glob(original_data_path+"*.xlsx")
zip_path = glob.glob(original_data_path+"*.zip")


## 1.1 이미지 데이터 통합

In [89]:
# zip파일 압축 해제 후 위치 변경
for zip in zip_path:
    zipfile.ZipFile(zip).extractall(path+'image')

In [90]:
# 이중 폴더 내에 있는 이미지 꺼내기
infolder_path = glob.glob(path+'image'+'/BA*')
for in_path in infolder_path:
    tmp = os.listdir(in_path)
    for i in tmp:
        # 파일 이름에서 앞의 0제거
        remove0_name = i.lstrip("0")

        #파일 이동 및 이름 변경
        os.rename(os.path.join(in_path, i), os.path.join(path+'image', remove0_name))

    os.rmdir(in_path)

In [91]:
# 남녀 이미지 분류
# 새로운 폴더 생성 후 복사
image_all = os.path.join(path, 'image')
image_M = os.path.join(path, 'image_M')
image_F = os.path.join(path, 'image_F')

# 출력 폴더 생성
os.makedirs(image_M, exist_ok=True)
os.makedirs(image_F, exist_ok=True)

# 이미지 파일 분류
image_files = glob.glob(os.path.join(image_all, '*.jpg'))

for image_file in image_files:
    # 파일 이름에서 Gender 추출
    gender = os.path.basename(image_file).split("_")[-1]

    # 새로운 경로 생성
    output_folder = image_M if gender == "M.jpg" else image_F
    output_path = os.path.join(output_folder, os.path.basename(image_file))

    # 이미지 복사
    shutil.copy(image_file, output_path)
    

## 1.2 Excel 데이터 통합

In [92]:
# 남/여 xlsx 파일 따로 읽어서 합치기
xlsx_path_M = '../BoneAge/Original_Data/BA_M/'
xlsx_path_F = '../BoneAge/Original_Data/BA_F/'
xlsx_M = glob.glob(xlsx_path_M + '*.xlsx')
xlsx_F = glob.glob(xlsx_path_F + '*.xlsx')

# 남/여 데이터프레임 생성
df_M = pd.DataFrame()
df_F = pd.DataFrame()

for xlsx_file in xlsx_M:
    xlsx_M = pd.read_excel(xlsx_file)
    df_M = pd.concat([df_M, xlsx_M], axis=0, ignore_index=True)

df_M.to_excel(path + 'df_M.xlsx')

for xlsx_file_f in xlsx_F:
    xlsx_F = pd.read_excel(xlsx_file_f)
    df_F = pd.concat([df_F, xlsx_F], axis=0, ignore_index=True)

df_F.to_excel(path + 'df_F.xlsx')


In [93]:
df_M.head()

,No.,Group,등록번호,생년월일,성별,진료의,검사 시 나이,신장,체중,BMI,처방일자,시행일자,BA 1,BA 2,Unnamed: 14
0,332_M,76,4173152,2002-04-17,1,2,9.950685,136.4,42.4,22.8,2012-03-27,2012-03-27,8.75,9.50,NaN
1,333_M,1439,7968480,2007-02-27,1,2,9.956164,148.9,44.3,20.0,2017-02-08,2017-02-08,12.50,12.25,NaN
2,334_M,1709,8269414,2008-03-07,1,2,9.964384,130.4,30.6,18.0,2017-08-16,2018-02-20,9.25,8.25,NaN
3,335_M,1710,8269417,2008-03-07,1,2,9.964384,128.1,26.2,16.0,2017-08-16,2018-02-20,9.25,8.25,NaN
4,336_M,364,5952933,2003-02-12,1,1,9.972603,128.7,32.7,19.8,2012-05-10,2013-01-30 00:00:00,11.00,11.25,NaN


In [94]:
df_F.head()

,No.,Group,등록번호,생년월일,성별,진료의,검사 시 나이,신장,체중,BMI,처방일자,시행일자,BA 1,BA 2
0,310,1698,8255049,2007-08-03,2,1,9.969863,129.5,26.9,16.1,2017-01-09,2017-07-20,9.75,9.75
1,311,1897,8537405,2008-08-22,2,1,9.989041,132.0,31.0,17.8,2018-02-28,2018-08-16,10.50,11.00
2,312,1422,7942635,2005-01-19,2,1,10.008219,136.4,33.2,17.9,2015-01-20,2015-01-20,11.00,11.25
3,313,1475,7995857,2005-02-09,2,1,10.049315,133.5,31.2,17.6,2015-02-25,2015-02-25,10.00,10.25
4,314,1888,8520261,2008-09-11,2,1,10.060274,130.6,23.7,13.9,2018-10-01,2018-10-01,10.00,9.75


In [95]:
# 남/여 No. 컬럼값 수정
# No. 값과 img 파일명이 동일하도록 수정
df_M['No.'] = df_M['No.'].apply(lambda x: str(int(re.sub(r'[^0-9]', '', str(x)))) + '_M')
df_F['No.'] = df_F['No.'].apply(lambda x: str(x) + '_F')

# 필요없는 컬럼 삭제
df_M.drop(columns=["Unnamed: 14"], inplace=True)

# 남/여 데이터프레임 합치기
df = pd.DataFrame()
df = pd.concat([df_M, df_F], axis=0).reset_index(drop=True)

# BA1, BA2 평균값 컬럼 생성
df['BA'] = (df['BA 1'] + df['BA 2']) / 2

# xlsx 파일로 저장
#df.to_excel(path + "df.xlsx")

# csv 파일로 저장
#df.to_csv(path + 'df.csv')


In [96]:
df.head()

,No.,Group,등록번호,생년월일,성별,진료의,검사 시 나이,신장,체중,BMI,처방일자,시행일자,BA 1,BA 2,BA
0,332_M,76,4173152,2002-04-17,1,2,9.950685,136.4,42.4,22.8,2012-03-27,2012-03-27,8.75,9.50,9.125
1,333_M,1439,7968480,2007-02-27,1,2,9.956164,148.9,44.3,20.0,2017-02-08,2017-02-08,12.50,12.25,12.375
2,334_M,1709,8269414,2008-03-07,1,2,9.964384,130.4,30.6,18.0,2017-08-16,2018-02-20,9.25,8.25,8.750
3,335_M,1710,8269417,2008-03-07,1,2,9.964384,128.1,26.2,16.0,2017-08-16,2018-02-20,9.25,8.25,8.750
4,336_M,364,5952933,2003-02-12,1,1,9.972603,128.7,32.7,19.8,2012-05-10,2013-01-30 00:00:00,11.00,11.25,11.125


In [97]:
# 중복된 No값 찾기
duplicate_no = df[df.duplicated("No.")]

if duplicate_no.empty:
    print("중복된 No값이 없습니다.")
else:
    print("중복된 값이 있습니다.")
    print(duplicate_no)


중복된 값이 있습니다.
        No.  Group     등록번호        생년월일  성별 진료의   검사 시 나이     신장    체중   BMI  \
519   185_M   1836  8433147  2010-07-12   1   2  6.983562  120.5  32.1  22.2   
1041  114_F   1278  7764132  2008-08-13   2   1  6.038356  108.5  17.7  15.1   
1195  268_F   1728  8294255  2007-05-30   2   1  9.186301  127.5  23.9  14.8   

            처방일자        시행일자  BA 1  BA 2     BA  
519   2017-07-04  2017-07-04  6.00   6.5  6.250  
1041  2014-01-27  2014-08-26  5.25   5.5  5.375  
1195  2016-08-03  2016-08-03  9.00   9.0  9.000  


In [98]:
#No가 중복된 행 확인하기
all_duplicate_rows_114_F = df[(df['No.'] == '114_F')]
print("No가 114_F인 모든 중복된 행:")
print(all_duplicate_rows_114_F)

all_duplicate_rows_268_F = df[(df['No.'] == '268_F')]
print("No가 268_F인 모든 중복된 행:")
print(all_duplicate_rows_268_F)

all_duplicate_rows_185_M = df[(df['No.'] == '185_M')]
print("No가 185_M인 모든 중복된 행:")
print(all_duplicate_rows_185_M)

No가 114_F인 모든 중복된 행:
        No.  Group     등록번호        생년월일  성별 진료의   검사 시 나이     신장    체중   BMI  \
1040  114_F    287  5919778  2005-04-18   2   1  5.931507  111.2  19.9  16.1   
1041  114_F   1278  7764132  2008-08-13   2   1  6.038356  108.5  17.7  15.1   

            처방일자        시행일자  BA 1  BA 2     BA  
1040  2011-03-23  2011-03-23  5.75  5.25  5.500  
1041  2014-01-27  2014-08-26  5.25  5.50  5.375  
No가 268_F인 모든 중복된 행:
        No.  Group     등록번호        생년월일  성별 진료의   검사 시 나이     신장    체중   BMI  \
1186  268_F   1699  8255297  2007-08-31   2   1  8.838356  128.9  25.7  15.5   
1195  268_F   1728  8294255  2007-05-30   2   1  9.186301  127.5  23.9  14.8   

            처방일자        시행일자  BA 1   BA 2      BA  
1186  2016-06-30  2016-06-30  10.0  10.25  10.125  
1195  2016-08-03  2016-08-03   9.0   9.00   9.000  
No가 185_M인 모든 중복된 행:
       No.  Group     등록번호        생년월일  성별 진료의   검사 시 나이     신장    체중   BMI  \
518  185_M   1943  8600937  2011-07-03   1   2  6.964384  124.4  26.1 

In [99]:
# No가 114_F 또는 268_F인 행 삭제
indexes_to_remove_114 = df[(df['No.'].isin(['114_F'])) & (df['BA'].isin([5.500]))].index
df.drop(indexes_to_remove_114, inplace=True)

indexes_to_remove_268 = df[(df['No.'].isin(['268_F'])) & (df['BA'].isin([10.125]))].index
df.drop(indexes_to_remove_268, inplace=True)

# No가 185_M인 행의 No.값을 186_M으로 변경
df.loc[(df['No.'] == '185_M') & (df['BA'].isin([6.25])), 'No.'] = '186_M'

# xlsx 파일로 저장
df.to_excel(path + "df_excel.xlsx")

#csv 파일로 저장
df.to_csv(path + 'df.csv')


# 2. train, test set 분리

In [100]:
df.head()

,No.,Group,등록번호,생년월일,성별,진료의,검사 시 나이,신장,체중,BMI,처방일자,시행일자,BA 1,BA 2,BA
0,332_M,76,4173152,2002-04-17,1,2,9.950685,136.4,42.4,22.8,2012-03-27,2012-03-27,8.75,9.50,9.125
1,333_M,1439,7968480,2007-02-27,1,2,9.956164,148.9,44.3,20.0,2017-02-08,2017-02-08,12.50,12.25,12.375
2,334_M,1709,8269414,2008-03-07,1,2,9.964384,130.4,30.6,18.0,2017-08-16,2018-02-20,9.25,8.25,8.750
3,335_M,1710,8269417,2008-03-07,1,2,9.964384,128.1,26.2,16.0,2017-08-16,2018-02-20,9.25,8.25,8.750
4,336_M,364,5952933,2003-02-12,1,1,9.972603,128.7,32.7,19.8,2012-05-10,2013-01-30 00:00:00,11.00,11.25,11.125


In [101]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 42, stratify = df['성별'])
train_df.to_csv(path + 'train_df.csv', index = 0)
test_df.to_csv(path + 'test_df.csv', index = 0)

In [102]:
train_id = train_df["No."].values.tolist()
test_id = test_df["No."].values.tolist()

# train_image와 test_image 폴더 생성
data_folder = '../BoneAge/Data/image/'
train_image_folder = '../BoneAge/Data/train_image/'
test_image_folder = '../BoneAge/Data/test_image/'

os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(test_image_folder, exist_ok=True)

In [103]:
# train 데이터의 이미지 이동
for user_id in train_id:
    source_path = os.path.join(data_folder, f'{user_id}.jpg')
    destination_path = os.path.join(train_image_folder, f'{user_id}.jpg')
    
    # 이미지 이동
    shutil.copy(source_path, destination_path)
    
# test 데이터의 이미지 이동
for user_id in test_id:
    source_path = os.path.join(data_folder, f'{user_id}.jpg')
    destination_path = os.path.join(test_image_folder, f'{user_id}.jpg')
    
    # 이미지 이동
    shutil.copy(source_path, destination_path)

# Thumb.db 라는 숨김파일이 있어 빈 폴더 지우는 코드로 지워지지 않음
# shutil.rmtree('../BoneAge/Data/', ignore_errors=True) # 폴더와 안의 내용을 강제로 지우는 코드